
# Work and Live in Houston

## IBM Data Science Professional Certificate • Capstone Project • Week 5 • Asadullah Aliyev


In [4]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.18.1-py_0 conda-forge

geographiclib- 100% |################################| Time: 0:00:00  22.88 MB/s
geopy-1.18.1-p 100% |################################| Time: 0:00:00  34.98 MB/s
Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  19.49 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  36.88 MB/s
vincent-0.4.4- 100% |###################

### Let's create empty dataframe *df* with a column named as Zip code.

In [5]:
df = pd.DataFrame(columns= ["Zip code"]) # empty dataframe
df

,Zip code


### Now let's extract Top 10 Zip codes from [Niche.com's](https://www.niche.com/places-to-live/search/best-zip-codes-to-buy-a-house/m/houston-metro-area/) "2018 Best ZIP Codes to Buy a House in the Houston Area" article and append to dataframe df.

In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

r= requests.get("https://www.niche.com/places-to-live/search/best-zip-codes-to-buy-a-house/m/houston-metro-area/")
soup = BeautifulSoup(r.content, "html.parser")

for i in range(10):  # Top 10 Zip codes
    df= df.append({'Zip code': int(soup.find_all("h2", {"class":"search-result__title"})[i].text)},ignore_index=True)

df

,Zip code
0,77345
1,77059
2,77094
3,77406
4,77546
5,77494
6,77479
7,77429
8,77586
9,77385


### [Best Places.net's]( https://www.bestplaces.net/) displays a breakdown of the crimes committed in the vicinity. Since there is not available listed bulk crime rates for Houston ZIP codes, I will use individual search funcionality offline for each case. I will use annual crime rate per 1000 resident indicator. 




In [7]:
crime_data = {'Zip code': [77345, 77059, 77094, 77406, 77546, 77494, 77479, 77429, 77586, 77385],
             'Crime rate': [38.3, 42.7, 34.7, 35.7, 50.7, 35.5, 32.5, 48.4, 70.2, 47.4]}
crime = pd.DataFrame(data=crime_data)
crime


,Crime rate,Zip code
0,38.3,77345
1,42.7,77059
2,34.7,77094
3,35.7,77406
4,50.7,77546
5,35.5,77494
6,32.5,77479
7,48.4,77429
8,70.2,77586
9,47.4,77385


### Now let's inner join df and crime dataframes under  dataframe *df*

In [8]:
df=pd.merge(df, crime, on='Zip code')
df

,Zip code,Crime rate
0,77345,38.3
1,77059,42.7
2,77094,34.7
3,77406,35.7
4,77546,50.7
5,77494,35.5
6,77479,32.5
7,77429,48.4
8,77586,70.2
9,77385,47.4


### For *"Area", "Travel to work","School rates","Closer to amenities" and "Home prices"*,  we will use comprehensive chart from the article written by Katharine Shilcutt in the April 2015 issue of [Houstonia magazine](https://www.houstoniamag.com/articles/2015/3/30/houston-neighborhoods-by-the-numbers-april-2015). Let's extract table headings from website.

In [9]:
r= requests.get("https://www.houstoniamag.com/articles/2015/3/30/houston-neighborhoods-by-the-numbers-april-2015")
soup = BeautifulSoup(r.content, "html.parser")

table_head=soup.find_all("table", {"id":"fixed_hdr"})[0].thead.find_all('tr')
table_head


[<tr>
 <th title="Field #1">Market Area</th>
 <th title="Field #2">ZIP Code</th>
 <th title="Field #3">2014 Median Home Price</th>
 <th title="Field #4">% Growth 2010-2014</th>
 <th title="Field #5">% Growth 2013-2014</th>
 <th title="Field #6">Avg. Days on Market in 2014</th>
 <th title="Field #7">Population</th>
 <th title="Field #8">% Owner Occupied</th>
 <th title="Field #9">Median Income</th>
 <th title="Field #10">Median Age</th>
 <th title="Field #11">% Below Poverty Line</th>
 <th title="Field #12">% Unemployed</th>
 <th title="Field #13">% of Population Under 14</th>
 <th title="Field #14">% Enrolled in Private Schools</th>
 <th title="Field #15">Grade for Elementary Schools</th>
 <th title="Field #16">Grade for Middle Schools</th>
 <th title="Field #17">Grade for High Schools</th>
 <th title="Field #18">Walkability Score</th>
 <th title="Field #19">% Using Public Transportation</th>
 <th title="Field #20">% Carpooling</th>
 <th title="Field #21">Mean Travel Time to Work</th>


### Then extract the columns we need and add them into newly dataframe *table*


In [10]:
table=pd.DataFrame(columns= []) # empty dataframe
table_row=soup.find_all("table", {"id":"fixed_hdr"})[0].tbody.find_all('td')

for i in range(len(table_row)-75): 
     if i % 25==0:  
        table= table.append({'Area': table_row[i].text, 
                             'Zip code': int(table_row[i+1].text),
                             'Travel to work': str(table_row[i+20].text).replace("<td>",""),
                             'School rates': str(table_row[i+16].text).replace("<td>",""),
                             'Closer to amenities': str(table_row[i+17].text).replace("<td>",""),
                             'Home prices-USD': str(table_row[i+2].text).replace("<td>","").replace("$","").replace(",","")
                            },ignore_index=True)

table.head()

,Area,Closer to amenities,Home prices-USD,School rates,Travel to work,Zip code
0,1960/Cypress,N/A,150000,N/A,27.07,77065.0
1,Aldine Area,N/A,112500,B+,29.53,77039.0
2,Alief,48,126750,B+,28.51,77072.0
3,Alvin North,23,193471,B+,30.36,77511.0
4,Alvin South,23,140000,B+,30.36,77511.0


### Now, let's merge (inner join) two dataframes table and df under df

In [11]:
df=pd.merge(df, table, on='Zip code')
df

,Zip code,Crime rate,Area,Closer to amenities,Home prices-USD,School rates,Travel to work
0,77345,38.3,Kingwood East,16,279900,N/A,35.96
1,77546,50.7,Friendswood,17,203094,B+,30.13
2,77494,35.5,Katy-Southwest,N/A,337500,B+,33.52
3,77479,32.5,Sugar Land South,24,442667,B+,29.96
4,77429,48.4,Cypress North,N/A,220000,B+,31.80


### As seen from  above table, only 5 items match in two dataframes. Now let's replace N/A items with average in "Close to amenities" field and B+ "School rates" field. Then make standardization to transform data columns into a common format which allows me to make the meaningful comparison 


In [12]:
df.loc[df['Closer to amenities']== 'N/A','Closer to amenities' ]=19
df.loc[df['School rates']== 'N/A','School rates' ]="B+"
df[["Closer to amenities", "Home prices-USD","Travel to work"]] = df[["Closer to amenities", "Home prices-USD","Travel to work"]].astype("float")
df["Crime rate"]=df["Crime rate"]/sum(df["Crime rate"])
df["Closer to amenities"]=df["Closer to amenities"]/sum(df["Closer to amenities"])
df["Home prices-USD"]=df["Home prices-USD"]/sum(df["Home prices-USD"])
df["Travel to work"]=df["Travel to work"]/sum(df["Travel to work"])
df["School rates"]=1
df

,Zip code,Crime rate,Area,Closer to amenities,Home prices-USD,School rates,Travel to work
0,77345,0.186465,Kingwood East,0.168421,0.188719,1,0.222842
1,77546,0.246835,Friendswood,0.178947,0.136933,1,0.186714
2,77494,0.172833,Katy-Southwest,0.200000,0.227555,1,0.207721
3,77479,0.158228,Sugar Land South,0.252632,0.298462,1,0.185660
4,77429,0.235638,Cypress North,0.200000,0.148332,1,0.197063


### We need latitude and logitude info per zips and we  will use [PeriscopeData.com](https://community.periscopedata.com/media/36x6sg/download/zip_codes_states.csv) info on this matter and add proper indicators to our dataframe 

In [13]:
zips=pd.read_csv("https://community.periscopedata.com/media/36x6sg/download/zip_codes_states.csv") # extract all US zip codes
zips= zips.rename(columns={'zip_code': 'Zip code'}) # rename column name zip_code for merge purposes
zips= zips[['Zip code','latitude','longitude']] # take the fields we need
df=pd.merge(df, zips, on='Zip code') # merge the dataframes
df

,Zip code,Crime rate,Area,Closer to amenities,Home prices-USD,School rates,Travel to work,latitude,longitude
0,77345,0.186465,Kingwood East,0.168421,0.188719,1,0.222842,29.833990,-95.434241
1,77546,0.246835,Friendswood,0.178947,0.136933,1,0.186714,29.482185,-95.156545
2,77494,0.172833,Katy-Southwest,0.200000,0.227555,1,0.207721,29.833990,-95.434241
3,77479,0.158228,Sugar Land South,0.252632,0.298462,1,0.185660,29.552921,-95.666778
4,77429,0.235638,Cypress North,0.200000,0.148332,1,0.197063,29.833990,-95.434241


### Let's create 'Magnitude' column to calculate square root of the sum of the components' squares. Since our values are inversely proportional, we will divide 1 to our result.

In [14]:
df['Magnitude']=np.sqrt(np.square(df[['Crime rate', 'Closer to amenities','Home prices-USD','Travel to work', 'School rates']].sum(axis=1)))
df['Magnitude']=1/(df['Magnitude']/sum(df['Magnitude']))
df


,Zip code,Crime rate,Area,Closer to amenities,Home prices-USD,School rates,Travel to work,latitude,longitude,Magnitude
0,77345,0.186465,Kingwood East,0.168421,0.188719,1,0.222842,29.833990,-95.434241,5.094973
1,77546,0.246835,Friendswood,0.178947,0.136933,1,0.186714,29.482185,-95.156545,5.144533
2,77494,0.172833,Katy-Southwest,0.200000,0.227555,1,0.207721,29.833990,-95.434241,4.977575
3,77479,0.158228,Sugar Land South,0.252632,0.298462,1,0.185660,29.552921,-95.666778,4.749387
4,77429,0.235638,Cypress North,0.200000,0.148332,1,0.197063,29.833990,-95.434241,5.053249


### Highest number means the best option. As shown from the table, Friendswood is the best neighborhood to buy a house. 

In [21]:
address = 'Houston'

geolocator = Nominatim(user_agent="tr_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Houston are {}, {}.'.format(latitude, longitude))

# create map of New York using latitude and longitude values
map_houston = folium.Map(location=[latitude, longitude], zoom_start=6)

# add markers to map
for lat, lng, zipcode in zip(df['latitude'], df['longitude'], df['Zip code']):
    label = '{}'.format('Zip code')
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_houston) 
 
    
map_houston

The geograpical coordinate of Houston are 29.7589382, -95.3676974.
